In [1]:
import numpy as np
import random as rd
import itertools
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [ ]:
# Suprime os warnings do pd.DataFrame.append()
import warnings
warnings.filterwarnings("ignore")

In [2]:
naipes = ['espadas', 'copas', 'ouros', 'paus']
numeros = ['A'] + [str(x) for x in range (2, 8)] + ['Q', 'J', 'K']
baralho_full = [num+'_'+naipe for num, naipe in list(itertools.product(numeros, naipes))]
rd.shuffle(baralho_full)

In [3]:
dic_tamanho = {0:0, '0':0, 'A_espadas': 12,
 'A_copas': 8,
 'A_ouros': 8,
 'A_paus': 8,
 '2_espadas': 9,
 '2_copas': 9,
 '2_ouros': 9,
 '2_paus': 9,
 '3_espadas': 10,
 '3_copas': 10,
 '3_ouros': 10,
 '3_paus': 10,
 '4_espadas': 1,
 '4_copas': 1,
 '4_ouros': 1,
 '4_paus': 14,
 '5_espadas': 2,
 '5_copas': 2,
 '5_ouros': 2,
 '5_paus': 2,
 '6_espadas': 3,
 '6_copas': 3,
 '6_ouros': 3,
 '6_paus': 3,
 '7_espadas': 4,
 '7_copas': 13,
 '7_ouros': 11,
 '7_paus': 4,
 'Q_espadas': 5,
 'Q_copas': 5,
 'Q_ouros': 5,
 'Q_paus': 5,
 'J_espadas': 6,
 'J_copas': 6,
 'J_ouros': 6,
 'J_paus': 6,
 'K_espadas': 7,
 'K_copas': 7,
 'K_ouros': 7,
 'K_paus': 7}

### Variáveis
- [OK] Valor da carta no slot 1 
- [OK] Valor da carta no slot 2
- [OK] Valor da carta no slot 3
- [OK] Placar da própria equipe
- [OK] Placar dos oponentes
- [OK] Carta jogada pelo p2 (0 se não jogou)
- [OK] Carta jogada pelo p3 (0 se não jogou)
- [OK] Carta jogada pelo p4 (0 se não jogou)
- [OK] Histórico de cartas que já apareceram
    - [OK] Uma bool pra cada carta do baralho
- [OK] Qtd de cartas na própria mão
- [OK] Qtd de cartas na mão do próximo jogador (p2)
- [OK] Qtd de cartas na mão do próximo jogador (p3)
- [OK] Qtd de cartas na mão do próximo jogador (p4)
- [OK] Número da rodada
- [OK] Quem fez a primeira (-1 se não terminou ainda)
- [OK] Quem fez a segunda (-1 se não terminou ainda)
- [OK] Valor do tento
- Se está trucando (definindo truco como dobrar a aposta) [Depois]
- Se vai trucar (definindo truco como dobrar a aposta) [Depois]
- [OK] Paridade (quem saiu)
- [OK] Paridade 2 (quem joga primeiro entre a dupla - 0 se for esse agente)
- Se está amarrado
- Quem fez a primeira (já fiz?)
- Se foi amarrado pelo parceiro
- Carta jogada pelo parceiro nessa rodada (0 se não jogou)

In [2]:
class tento:
    def __init__(self, start, queda):
        self.verbose = queda.verbose
        self.baralho = baralho_full
        rd.shuffle(self.baralho)
        self.p1 = self.baralho[1:4]
        self.p2 = self.baralho[4:7]
        self.p3 = self.baralho[7:10]
        self.p4 = self.baralho[10:13]
        self.rodada = 0
        self.valor = 2
        self.players = [self.p1, self.p2, self.p3, self.p4]
        self.start = start
        self.queda = queda
        self.cartas_jogadas = set()
        self.primeira = -1
        self.segunda = -1
        self.mesa = []
        
    def is_amarrado(self, lista):
        try:
            return max(lista[::2]) == max(lista[1::2])
        except:
            return False
    
    def jogar(self, p_number, mesa, df=df_replay):
        # Pegando a var global do exp replay
        global df_replay
        # Aqui precisa inserir a função de decisão
        # Vai precisar inserir uns get_states() pra pegar as variáveis
        p = self.players[p_number]
        # Construindo variáveis de estado
        dic_vars = {
        'valor_carta_1':dic_tamanho[p[0]],
        'valor_carta_2':dic_tamanho[p[1]],
        'valor_carta_3':dic_tamanho[p[2]],
        'placar_proprio':self.queda.placar[p_number%2],
        'placar_oponente':self.queda.placar[p_number%2],
        'qtd_jogadas':len(mesa)}
        
        dic_aux = {
        'carta_p1':dic_tamanho[mesa[0]] if dic_vars['qtd_jogadas']==3 else 0,
        'carta_p2':dic_tamanho[mesa[1]] if dic_vars['qtd_jogadas']==3 else dic_tamanho[mesa[0]] if dic_vars['qtd_jogadas']==2 else 0,
        'carta_p3':dic_tamanho[mesa[2]] if dic_vars['qtd_jogadas']==3 else dic_tamanho[mesa[1]] if dic_vars['qtd_jogadas']==2\
                                             else dic_tamanho[mesa[0]] if dic_vars['qtd_jogadas']==1 else 0,
        'qtd_cartas_propria_mao':sum([valor > 0 for valor in [dic_tamanho[carta] for carta in p]]),
        'qtd_cartas_p1':sum([valor > 0 for valor in [dic_tamanho[carta] for carta in filter(lambda c: c!=0, self.players[(p_number+1)%4])]])   ,
        'qtd_cartas_p2':sum([valor > 0 for valor in [dic_tamanho[carta] for carta in filter(lambda c: c!=0, self.players[(p_number+2)%4])]]),
        'qtd_cartas_p3':sum([valor > 0 for valor in [dic_tamanho[carta] for carta in filter(lambda c: c!=0, self.players[(p_number+3)%4])]]),
        'valor_tento':self.valor,
        'num_rodada':self.rodada,
        'primeira':self.primeira,
        'segunda':self.segunda,
        'paridade':self.start,
        'paridade_dupla':0 if len(self.mesa) < 2 else 1,
        'amarrado':self.is_amarrado(self.mesa),
        #amarrado_parceiro = 
        #index_carta = agent.jogar() #Passar variáveis de status aqui
        'carta_escolhida':rd.choice(list(filter(lambda c: c!=0, p)))}
        
        dic_vars.update(dic_aux)
        
        dic_bool_cartas = {carta:int(carta in self.cartas_jogadas)\
                           for carta in set(dic_tamanho.keys()) - {0, '0'}}
        
        index_carta = p.index(dic_vars['carta_escolhida'])
        carta_jogada = p[index_carta]
        
        df_replay = df_replay.append(dic_vars | dic_bool_cartas, ignore_index=True)
        p[index_carta] = 0
        self.cartas_jogadas.add(carta_jogada)
        #print(carta_jogada)
        return carta_jogada
        
    def jogar_mao(self, start=0):
        mesa = []
        for i in range(start, start+4):
            mesa.append(self.jogar(p_number=i%4, mesa=mesa))
            self.mesa = mesa
        if self.verbose:
            print('Mesa: \n', mesa)
        return mesa
    
    def jogar_tento(self):
        torna = self.start
        dic_vitorias_maos = {}
        while self.rodada < 3:
            mesa = self.jogar_mao(start=torna)
            mesa_tamanhos = [dic_tamanho[c] for c in mesa]
            amarrado = self.is_amarrado(mesa_tamanhos)
            if amarrado:
                for i in range(1, 4):
                    if max(mesa_tamanhos) == mesa_tamanhos[i]:
                        next_torna = i
                dic_vitorias_maos.update({self.rodada:-1})
            else:
                next_torna = (mesa_tamanhos.index(max(mesa_tamanhos))+torna)%4
                if self.rodada == 1:
                    self.primeira = next_torna%2
                elif self.rodada == 2:
                    self.segunda = next_torna%2
                dic_vitorias_maos.update({self.rodada:next_torna%2})
            torna = next_torna
            self.rodada += 1
        if dic_vitorias_maos[0] == -1:
            if dic_vitorias_maos[1] != -1:
                if self.verbose:
                    print('[Amarrou 1ª] Dupla {} vitoriosa!'.format(dic_vitorias_maos[1]))
                return dic_vitorias_maos[1], self.valor
            else:
                if self.verbose:
                    print('[Amarrou 1ª e 2ª] Dupla {} vitoriosa!'.format(dic_vitorias_maos[2]))
                return dic_vitorias_maos[2], self.valor
        elif (dic_vitorias_maos[1] == -1) or (dic_vitorias_maos[2] == -1):
            if self.verbose:
                print('[Amarrou 2ª ou 3ª] Dupla {} vitoriosa!'.format(dic_vitorias_maos[0]))
            return dic_vitorias_maos[0], self.valor
        else:
            if list(dic_vitorias_maos.values()).count(1) >= 2:
                if self.verbose:
                    print('[Não amarrou] Dupla 0 vitoriosa!')
                return 1, self.valor
            if list(dic_vitorias_maos.values()).count(0) >= 2:
                if self.verbose:
                    print('[Não amarrou] Dupla 1 vitoriosa!')
                return 0, self.valor
        if self.verbose:
            print(self.cartas_jogadas)
            
    def maos(self):
        print('p1:', self.p1)
        print('p2:', self.p2)
        print('p3:', self.p3)
        print('p4:', self.p4)

IndentationError: unexpected indent (1753736103.py, line 71)

In [288]:
class queda:
    def __init__(self, verbose=False):
        self.placar = {0:0, 1:0}
        self.verbose = verbose
        
    def jogar_queda(self):
        dupla_que_inicia = round(rd.random())
        while not ((self.placar[0] >= 12) or (self.placar[1] >= 12)):
            dupla_que_inicia = int(not dupla_que_inicia)
            t = tento(start=dupla_que_inicia, queda=self)
            dupla, pontos = t.jogar_tento()
            if dupla != -1:
                self.placar[dupla] += pontos
        return self.placar            

In [296]:
model_vars = ['valor_carta_1',  'valor_carta_2',  'valor_carta_3',  'placar_proprio',  'placar_oponente',  'qtd_jogadas',  'carta_p1',  'carta_p2',  'carta_p3',  'qtd_cartas_propria_mao',  'qtd_cartas_p1',  'qtd_cartas_p2',  'qtd_cartas_p3',  'valor_tento',  'num_rodada',  'primeira',  'segunda',  'paridade',  'paridade_dupla',  'amarrado',  'carta_escolhida',  'index_carta',  'carta_jogada']
df_replay = pd.DataFrame(columns=model_vars+list(set(dic_tamanho.keys() - {0, '0'})))

In [297]:
%%time
for i in range(100):
    queda().jogar_queda()

CPU times: total: 2min 40s
Wall time: 2min 40s


In [298]:
df_replay

,valor_carta_1,valor_carta_2,valor_carta_3,placar_proprio,placar_oponente,qtd_jogadas,carta_p1,carta_p2,carta_p3,qtd_cartas_propria_mao,...,J_espadas,J_paus,5_ouros,4_ouros,2_copas,A_ouros,6_paus,J_ouros,K_ouros,K_paus
0,11,1,1,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
1,7,2,2,0,0,1,0,0,1,3,...,0,0,0,0,0,0,0,0,0,0
2,5,6,8,0,0,2,0,1,7,3,...,0,0,0,0,0,0,0,0,0,0
3,2,6,3,0,0,3,1,7,6,3,...,0,1,0,0,0,0,0,0,0,0
4,0,2,2,0,0,0,0,0,0,2,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10951,5,4,0,6,6,3,9,9,2,2,...,0,0,0,0,1,0,1,0,0,0
10952,0,0,6,6,6,0,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0
10953,7,0,0,10,10,1,0,0,6,1,...,1,0,0,0,1,0,1,0,0,0
10954,0,4,0,6,6,2,0,6,7,1,...,1,0,0,0,1,0,1,0,0,0


In [283]:
%%time
df = []
for i in range(1, 1):
    resultado = [list(queda().jogar_queda().values())]
    df = df + resultado
df = pd.DataFrame(df)
df.columns = ['d0', 'd1']

ValueError: Length mismatch: Expected axis has 0 elements, new values have 2 elements

In [267]:
df_replay

,p,valor_carta_1,valor_carta_2,valor_carta_3,placar_proprio,placar_oponente,qtd_jogadas,carta_p1,carta_p2,carta_p3,...,J_espadas,J_paus,5_ouros,4_ouros,2_copas,A_ouros,6_paus,J_ouros,K_ouros,K_paus
0,NaN,10,9,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,2,1,11,0,0,1,0,0,6,...,0,0,0,0,0,0,0,0,0,0
2,NaN,9,13,10,0,0,2,0,6,2,...,0,0,0,0,0,0,0,0,0,0
3,NaN,7,8,8,0,0,3,6,2,13,...,0,0,0,0,0,0,0,0,0,0
4,NaN,9,0,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,NaN,14,1,0,8,8,3,1,4,12,...,0,0,0,1,0,0,1,0,0,0
1052,NaN,0,0,9,10,10,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1053,NaN,14,0,0,8,8,1,0,0,9,...,0,0,0,1,1,0,1,0,0,0
1054,NaN,8,0,0,10,10,2,0,9,14,...,0,0,0,1,1,0,1,0,0,0


{0: 6, 1: 12}

In [ ]:
# Pra jogar é necessário inicializar um dataframe df_replay

In [24]:
# Proporção de vitórias das duplas
np.std(df['d0'] < df['d1'])

0.49964796174271775